# Load das APIs

In [1]:
from src.api import Client
api = Client('dev')

> Inicializando API Client em=dev
> Utilizando o token=token_dummy_001


# Train

In [2]:
!pip install statsmodels
!pip install pyarrow
!pip install scikit-learn
!pip install pycaret
!pip install hurst
!pip install cuml

  Using cached cuml-0.6.1.post1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cuml
  Running setup.py clean for cuml
Failed to build cuml
ERROR: Could not build wheels for cuml, which is required to install pyproject.toml-based projects


In [3]:
"""
from src.simple_robot import SimpleRobot

simple_robot = SimpleRobot(api)
simple_robot.train()
"""

from src.pycaret_robot import PycaretRobot

pycaret_robot = PycaretRobot(api)
pycaret_robot.train()


>>> PycaretRobot

>>> Etapa Treinamento
... carregando treinamento

>>> Feature Eng Entrada=Index(['symbol', 'datetime', 'open', 'high', 'low', 'close', 'volume',
       'number_of_trades'],
      dtype='object')
... Trabalhando a partir dos dados de 2023-09-22 10:38:00
>>> Feature Eng Saída=Index(['symbol', 'open', 'high', 'low', 'close', 'volume', 'number_of_trades',
       'value_variation', 'value_class', 'year', 'month', 'day', 'hour',
       'minute', 'week_day', 'traded_volume', 'amplitude', 'candle',
       'h_value_variation', 'h_traded_volume', 'h_amplitude', 'h_candle'],
      dtype='object')

... debug de features salvo em = data/temp.features.csv
... setup


,Description,Value
0,Session id,123
1,Target,value_class
2,Target type,Multiclass
3,Target mapping,"negative: 0, negative_soft: 1, negative_strong: 2, no_variation: 3, positive: 4, positive_soft: 5, positive_strong: 6"
4,Original data shape,"(1737, 22)"
5,Transformed data shape,"(1737, 45)"
6,Transformed train set shape,"(1389, 45)"
7,Transformed test set shape,"(348, 45)"
8,Ordinal features,1
9,Numeric features,17


... compare models


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.9978,0.9988,0.9978,0.9981,0.9979,0.9974,0.9975,0.1520
gbc,Gradient Boosting Classifier,0.9978,0.9997,0.9978,0.9981,0.9979,0.9974,0.9975,4.7990
xgboost,Extreme Gradient Boosting,0.9978,1.0000,0.9978,0.9981,0.9979,0.9974,0.9975,1.0500
lightgbm,Light Gradient Boosting Machine,0.9957,0.9999,0.9957,0.9960,0.9957,0.9949,0.9949,3.7760
rf,Random Forest Classifier,0.9914,0.9997,0.9914,0.9916,0.9913,0.9897,0.9898,0.8020
et,Extra Trees Classifier,0.8265,0.9762,0.8265,0.8285,0.8222,0.7926,0.7937,0.3970
ada,Ada Boost Classifier,0.7142,0.9235,0.7142,0.5493,0.6099,0.6476,0.6797,0.3570
nb,Naive Bayes,0.6609,0.9017,0.6609,0.6973,0.6609,0.6019,0.6075,0.1410
lr,Logistic Regression,0.5767,0.8656,0.5767,0.5787,0.5550,0.4898,0.4963,1.5620
qda,Quadratic Discriminant Analysis,0.5659,0.8766,0.5659,0.6168,0.5564,0.4869,0.5025,0.1520


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

... best model
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       random_state=123, splitter='best')


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,0.9856,0.9919,0.9856,0.9873,0.9857,0.9829,0.9831
7,0.9928,0.9959,0.9928,0.9932,0.9928,0.9914,0.9915
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Processing:   0%|          | 0/4 [00:00<?, ?it/s]


>>> Avaliação de modelo


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

# Robo

In [4]:
#from src.pycaret_robot import PycaretRobot

#minutos = 1 # 1 trade a cada 1 min
#pycaret_robot = PycaretRobot(api)
#pycaret_robot.execute(minutos)